TransFormer는 기본적으로 Sequential Data를 다룬다

### Sequential Model
- Sequential modeling의 문제점
    - Trimmed sequence : 잘림
    - Omitted sequence : 중간이 생략
    - Permuted sequence : 순서가 섞임

## Transformer
- Transformer is the first sequence transduction model based entirely on attention
- RNN은 재귀적으로 돌아갔었지만 transformer는 attention을 사용하였다


- Transformer는 Sequential 한 데이터를 처리하고 인코딩 하므로 NMT(신경망 기계번역)문제에만 적용되지않고 
- Image Classfication, detection, DALL-E(문장에 맞는 이미지 생성. 정확히는 gpt-3지만 깊숙한원리는 Transformer) 등에도 사용할 수 있다.

<img src="../images/tf.png" width = 300>

- 예시) 독일어를 영어로 번역


<img src="../images/tf2.png" width = 500>

- RNN은 3개의 단어가 들어가면 3번 재귀적으로 돌아갔지만
- Transformer의 self attention구조는 n개의 단어를 한번에 처리할 수 있다.
- 동일한 구조를 갖지만 다른 parameter를 가진 Encoder와 Decoder가 있다.

Q?
1. N개의 단어가 어떻게 처리가 되는지
2. Encoder와 Decoder사이에 어떤 정보를 주고받는지
3. Decoder가 어떻게 generalization 할 수있는지

## Encoder

<img src="../images/selfat.png" width = 500>

- self attention이 핵심
- 3개의 단어가 주어지면 3개의 vector를 찾아준다
- 단어를 $x_i$를 $\vec z_i$로 바꿀 때 나머지 n-1개를 고려한다
    - self attention의 가장 큰 의미
    - 서로 dependencies가 있다

- Feed-forward 는 independent

- **Self-Attention** at high level
    - 아래 문장이 주어졌을 때 `it`은 어떤 단어들과 dependent한가?  
        : `animal`
    - `The animal didn't cross the street because it was too tired`


<img src="../images/tf3.png" width = 500>
- 간단한 두 단어 **Thinking**과 **Machines**이 주어졌다고 하면


<img src="../images/tf4.png" width = 500>

- 한개의 단어에 대해 self attention은 3가지 vector를 만들어낸다
    - Queries 
        - $q_i$
        - 영향을 받는 단어
        
    - Keys  
        - $k_i$
        - 영향을 주는 단어
    - Values 
        - $v_i$
        - 영향에 대한 가중치
    
- 이 3가지 vector를 통해 Embedding vector $x_i$를 새로운 vector로 바꾼다

<img src="../images/tf6.png" width = 500>

- Score : i 번째 Query와 모든 n개의 keys를 내적한다
    - $q_1\cdot k_1$ $q_1\cdot k_2$...
    - i번째 단어와 나머지 단어간의 interaction을 학습하게 해준다
    

- Normalization
    - Divide by 8(hyperparameter) : $d_k$, key의 demension에 dependent
    - softmax를 취해준다


- final encoding is done by the weighted sum of the vaule vectors
    - softmax를 취해준 Scores와 Value를 가중합 한 것이 최종적으로 구할
    - 인코딩된 벡터 $z_i$ 이다.

이 과정을 행렬로 한번에 표현할 수 있다. PyTorch로 1~2줄로 표현 가능  



왜 잘 될까?
- Image을 cnn,mlp로 dimension을 바꾸면 input이 고정된다.
- Transformer는 하나의 input이 고정되어도 옆의 단어들에 따라 출력이 달라진다
- 더 flexible한 구조

<img src="../images/tf13.png" width = 500>


### MHA : Multi-headed attention

<img src="../images/tf7.png" width = 500>

- Query를 1개가 아닌 n개 만든다(Multi-head)
    - 1개의 파라미터로 모델이 충분한 복잡성을 못 가질 수도 있기 때문
    - 여러번 Linear transform 시켜준다
    - 학습 파라미터 수를 늘려 모델의 복잡도를 올리는 방법


<img src="../images/tf8.png" width = 500>

- n개의 attention을 반복하면 n개의 encoding된 vector가 나온다

<img src="../images/tf9.png" width = 500>

- x, z의 차원을 맞춰줘아한다
    - 10 차원 벡터 x로 8개의 z 벡터를 만들면 80x10 행렬을 곱해 줄인다


- 실제로는 100개의 차원이면 10로 나눠서 Q,K,V를 만든다(?)


**Scaled Dot-Product Attention**
$$\text{Attention}(Q,K,V)=\text{softmax}(\frac{QK^t}{\sqrt{d_k}})V$$

**Multi-Head Attention**

$$\text{MultiHead(Q,K,V)}=\text{Concat}(\text{head}_1,\dots,\text{head}_h)W^O$$
$$\text{where head}_i = \text{Attention}(QW^Q_i,KW^K_i,VW^V_i)$$

### Positional Encoding

<img src="../images/tf12.png" width = 500>

#### <center>sin, cos 파도모양의 그래프 형태<center>

$pos$ : 문장에서 임베딩 벡터의 위치    
$i$ : 인덱스  
$d_{model}$ : 모든 층의 output vector (hyperparameter)
    
$$\text{짝수 : } PE(pos,2i) = \sin(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$$


$$\text{홀수 : } PE(pos,2i+1) = \cos(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$$


`position = torch.arange(max_len).unsqueeze(1)`  
`div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))`  
`pe = torch.zeros(max_len, 1, d_model)`  
`pe[:, 0, 0::2] = torch.sin(position * div_term)`  
`pe[:, 0, 1::2] = torch.cos(position * div_term)`   
    
- a,b,c를 넣든 b,c,a를 넣든 순서가 고려되지 않아 같은 결과가 나온다
- embedding 벡터 $x_i$에 값을 더해준다


## Decoder

- Encoder는 input을 표현하고 Decoder는 생성한다
- Encoder에서 Decoder에 어떤 값을 주는지?

<img src="../images/tf10.png" width = 500>

- Encoder에서 Decoder에 Key와 Value를 보낸다
- Autoregressive하게 단어를 만들어낸다


- **masking** : 문장을 다 알고 있으면 학습 의미가 없으므로 masking 해준다
    - 이전 단어들에만 dependent하게
    - 미래의 정보를 활용하지않는다
    

<img src="../images/tf11.png" width = 500>
- 마지막으로 단어들의 분포를 만들어 samling 한다?